In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
# ANDREA CAMBIA ESTO
CONTEO_MINIMO = 1

### -------------------------------------------------------------------------------------------------------------------------------

In [14]:
comedicaciones_dir = "2020_covid_comedicaciones/"

In [15]:
comedicaciones = pd.read_excel("co-medicaciones.xlsx")
comedicaciones = comedicaciones[comedicaciones.medicamento != comedicaciones.medicamento_principal.str.lower()]
comedicaciones['comeds'] = comedicaciones.medicamento.str.lower() + "_" + comedicaciones.medicamento_principal.str.lower()

def order_comeds(comeds):
    ordering = comeds.split("_")
    ordering.sort()
    return "_".join(ordering)

comedicaciones['comeds'] = comedicaciones.comeds.apply(lambda x: order_comeds(x))
comedicaciones = comedicaciones[["comeds", "conteo"]]
comedicaciones = comedicaciones.groupby('comeds').sum()
comedicaciones = comedicaciones.reset_index()

In [16]:
comedicaciones = comedicaciones[comedicaciones.conteo >= 2].sort_values(by="conteo", ascending=False)

In [17]:
def join_and_filter_by_keyword(df, keyword):
    df["unidos"] = ""
    for col in df.columns:
        df["unidos"] = df["unidos"].values + " " + df[col].astype(str).values
        
    df["unidos"] = df.unidos.str.lower()

    df = df[df.unidos.str.contains(keyword)]
    return df

In [18]:
# ANDREA - YA QUE AGREGES LOS ARCHIVOS VACIA LA LISTA (IGNORE_LIST = [])
IGNORE_LIST =  []

In [19]:
def get_combined_df(comeds_keyword):
    medicamento1, medicamento2 = comeds_keyword.split("_")
    if medicamento1 in IGNORE_LIST or medicamento2 in IGNORE_LIST:
        return pd.DataFrame()
    excel_filename1 = medicamento1.upper() + "_comedicaciones.xlsx"
    excel_filename1 = os.path.join(comedicaciones_dir, excel_filename1)

    med1_df = pd.read_excel(excel_filename1)
    med1_df = join_and_filter_by_keyword(med1_df, medicamento2)
        
    excel_filename2 = medicamento2.upper() + "_comedicaciones.xlsx"
    excel_filename2 = os.path.join(comedicaciones_dir, excel_filename2)

    med2_df = pd.read_excel(excel_filename2)
    med2_df = join_and_filter_by_keyword(med2_df, medicamento1)
    
    final_df = med1_df.append(med2_df)
    final_df = final_df.drop_duplicates(subset="Report Id")
    return final_df

In [20]:
# def get_real_count(comeds):
#     temp_df = get_combined_df(comeds)
#     return temp_df.shape[0]

# comedicaciones["real_count"] = comedicaciones.comeds.apply(lambda x: get_real_count(x))

# comedicaciones = comedicaciones[comedicaciones.real_count >= CONTEO_MINIMO].sort_values(by="real_count", ascending=False)

In [21]:
def get_statistics(df):
    print("Number of Cases: {}\n".format(df.shape[0]))
    print("Country Counts:")
    print(final_df["Reporter Country"].value_counts())
    
    print("\nAge Group Counts:")
    print(final_df['Age Group'].value_counts())

### -------------------------------------------------------------------------------------------------------------------------------

In [3]:
comedicaciones

NameError: name 'comedicaciones' is not defined

In [23]:
comedicaciones.to_excel("comedicaciones_list.xlsx", index = False)

In [24]:
# ANDREA PON EL NOMBRE DE LOS COMEDS AQUI
COMED_KEYWORD = "hydroxychloroquine_lopinavir-ritonavir"

In [25]:
final_df = get_combined_df(COMED_KEYWORD)

In [26]:
get_statistics(final_df)

Number of Cases: 43

Country Counts:
Italy          17
Spain           9
Iran            6
USA             2
South Korea     2
Switzerland     1
Bahrain         1
Iraq            1
Brunei          1
Qatar           1
Name: Reporter Country, dtype: int64

Age Group Counts:
Elderly        17
Adolescents     1
Children        1
Infants         1
Name: Age Group, dtype: int64


In [10]:
commorbilities = pd.read_excel("Relación muertes-fármaco.xlsx")

In [3]:
commorbilities_counts = commorbilities["COMMORBILITIES"].value_counts()

In [4]:
commorbilities_counts.to_excel("commorbility_count.xlsx", index=True)

In [5]:
deaths = pd.read_excel("Relación muertes-fármaco.xlsx")

In [6]:
deaths_counts = deaths["REASON OF DEAD"].value_counts()

In [7]:
deaths_counts.to_excel("deaths_count.xlsx", index=True)

In [19]:
deaths

,REPORT,PATIENT/NUMBER OF CASES,DRUG,RECEIVED DRUGS,COMMORBILITIES,AGE,REASON OF DEAD,REPORTED COUNTRY
0,803497471,CASE 1,PROPOFOL,"CASE 1: oseltamivir, ceftriaxone, hydroxychlor...",Pregnancy,38-year-old pregnant woman,cardiac arrest,IRAN
1,803494953,CASE 1,PROPOFOL,"fentanyl, midazolam, propofol, and neuromuscu...",hypertension,73 year-old man,SARS-CoV-2 infection with acute respiratory,USA
2,803503289,CASE 1,PIPERACILINA-TAZOBACTAM,"azithromycin, hydroxychloroquine, piperacilina...",relapse,51 year-old - woman,cause of death not stated,USA
3,803502515,CASE 1,PIPERACILINA-TAZOBACTAM,piperacillin/tazobactam and darunavir.,"hypertension, dyslipidaemia and chronic kidney...",75 year-old woman,cause of death not stated,ITALY
4,803502515,CASE 1,PIPERACILINA-TAZOBACTAM,piperacillin/tazobactam and darunavir.,chronic kidney disease,75 year-old woman,cause of death not stated,ITALY
...,...,...,...,...,...,...,...,...
106,803487115,3 CASES,TOCILIZUMAB,tocilizumab and methylprednisolone,hypertension,15 patients (12 men and 3 women) aged 62−80 ye...,cause of death not stated,CHINA
107,803486941,CASE 1,TOCILIZUMAB,azithromycin and meropenem + hydroxychloroquin...,not specified,68-year-old man,septic shock,GREECE
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
medicamentos = pd.read_excel("Relación muertes-fármaco_original.xlsx")

In [9]:
medicamentos_muertes_counts = medicamentos["DRUG"].value_counts()

In [57]:
medicamentos_muertes_counts

HIDROXICLOROQUINA          23
NOREPINEFRINA              11
LOPINAVIR                  10
PIPERACILINA-TAZOBACTAM     7
DEXAMETHASONE               6
MEROPENEM                   5
OSELTALMIVIR                5
TOCILIZUMAB                 4
CHLOROQUINE                 2
HEPARINA                    2
PROPOFOL                    2
LINEZOLID                   1
TRIMETOPRIM                 1
REMDESIVIIR                 1
CEFTRIAXONE                 1
MIDAZOLAM                   1
Name: DRUG (carpeta del fármaco en la que se encontró el reporte), dtype: int64

In [23]:
medicamentos_muertes_counts.to_excel("drug_death_count.xlsx", index=True)

In [12]:
commorbilities[commorbilities.DRUG == "OSELTALMIVIR"].REASON_OF_DEATH.value_counts()

AttributeError: 'DataFrame' object has no attribute 'REASON_OF_DEATH'